In [104]:
import glob 
import os
import cv2
import llama_index
import PIL


In [3]:
os.getcwd()

'/Users/Fritz/Workspace/Yale_Visiting_Day_Poster'

In [90]:
def find_data(CK=True, JAFFE=True, FER2013=True):

    emo_dict = {"anger":0, "contempt":0, "disgust":0, "fear":0, "happiness":0, "neutral":0, "sadness":0, "surprise":0}
    list_paths = []

    # CK+ load
    if CK:
        for emotion in ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]:
            emo_folder_path = "../Polygence/Datasets/CK+/" + emotion
            for pic_file in glob.glob(os.path.join(emo_folder_path, "*png")):
                list_paths.append([emotion, pic_file])

    # JAFFE load
    if JAFFE:
        for pic_file in glob.glob(os.path.join("../Polygence/Datasets/JAFFE", "*tiff")):
            if "AN" in pic_file[26:]:
                list_paths.append(["anger", pic_file])
            elif "DI" in pic_file[26:]:
                list_paths.append(["disgust", pic_file])
            elif "FE" in pic_file[26:]:
                list_paths.append(["fear", pic_file])
            elif "HA" in pic_file[26:]:
                list_paths.append(["happiness", pic_file])
            elif "NE" in pic_file[26:]:
                list_paths.append(["neutral", pic_file])
            elif "SA" in pic_file[26:]:
                list_paths.append(["sadness", pic_file])
            else:
                list_paths.append(["surprise", pic_file])

    #FER-2013
    if FER2013:
        FER_emo_dict = {0:"anger", 1:"disgust", 2:"fear", 3:"happiness", 4:"neutral", 5:"sadness", 6:"surprise"}
        for set in ["test/", "train/"]:
            root = "../Polygence/Datasets/FER2013/" + set
            for index, emotion in enumerate(["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]):
                emo_folder_path = root + emotion
                for pic_file in glob.glob(os.path.join(emo_folder_path, "*jpg")):
                    list_paths.append([FER_emo_dict[index], pic_file])   
    for (a,b) in list_paths:
        emo_dict[a] +=1
    print(emo_dict)
    return list_paths


In [19]:
#print(list_paths)
print(len(list_paths))
print(list_paths[0])
print(list_paths[0:50])

213
['anger', '../Polygence/Datasets/JAFFE/MK.AN3.127.tiff']
[['anger', '../Polygence/Datasets/JAFFE/MK.AN3.127.tiff'], ['fear', '../Polygence/Datasets/JAFFE/MK.NE1.113.tiff'], ['anger', '../Polygence/Datasets/JAFFE/YM.AN3.63.tiff'], ['fear', '../Polygence/Datasets/JAFFE/MK.SA1.119.tiff'], ['anger', '../Polygence/Datasets/JAFFE/KA.AN1.39.tiff'], ['fear', '../Polygence/Datasets/JAFFE/KA.SU2.37.tiff'], ['fear', '../Polygence/Datasets/JAFFE/YM.NE2.50.tiff'], ['fear', '../Polygence/Datasets/JAFFE/TM.HA2.181.tiff'], ['fear', '../Polygence/Datasets/JAFFE/NA.FE2.218.tiff'], ['fear', '../Polygence/Datasets/JAFFE/YM.NE3.51.tiff'], ['disgust', '../Polygence/Datasets/JAFFE/MK.DI1.128.tiff'], ['anger', '../Polygence/Datasets/JAFFE/YM.AN2.62.tiff'], ['anger', '../Polygence/Datasets/JAFFE/KL.AN2.168.tiff'], ['fear', '../Polygence/Datasets/JAFFE/YM.HA2.53.tiff'], ['anger', '../Polygence/Datasets/JAFFE/NA.AN2.212.tiff'], ['fear', '../Polygence/Datasets/JAFFE/KM.SA5.13.tiff'], ['fear', '../Polygence/Da

In [6]:
# for idx, [emo, file] in enumerate(list_paths):
#         image = cv2.imread(file, 0)
#         # Convert the BGR image to RGB before processing.
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [102]:
# os.environ['OPENAI_API_KEY']=### os.environ['OPENAI_API_BASE']=##
from langchain_community.llms import Ollama
from langchain.schema import HumanMessage, SystemMessage
import base64
from io import BytesIO

from IPython.display import HTML, display
from PIL import Image


llm = Ollama(model="bakllava")

def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Re-sized Base64 string
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="tiff")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def plt_img_base64(img_base64):
    """
    Display base64 encoded string as image

    :param img_base64:  Base64 string
    """
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


file_path = list_paths[150][1]
print(list_paths[150][0])
pil_image = Image.open(file_path)
image_b64 = convert_to_base64(pil_image)
#plt_img_base64(image_b64)

llm_with_image_context = llm.bind(images=[image_b64])
llm_with_image_context.invoke("What is the facial emotion expressed")

# messages = [
#     SystemMessage(
#         content="You are a helpful assistant that translates English to French."
#     ),
#     HumanMessage(content="I love programming."),
# ]
# # print(str(messages))
# llm(str(messages))

fear


ValueError: Ollama call failed with status code 400. Details: {"error":"unsupported image format"}

In [116]:
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def classify_emotion(file_path, display=False):
    output_parser = StrOutputParser()
    pil_image = Image.open(file_path)
    image_b64 = convert_to_base64(pil_image)
    if display:
        plt_img_base64(image_b64)

    llm = ChatOllama(model="bakllava")

    messages = [
        SystemMessage(
            content="You are going to recognize the facial emotion of the given picture. Here is some guiding information. Anger is communicated by lowered eyebrows, widened eyes, firmly pressed lips, and flared nostrils. Disgust brings a wrinkled nose, raised upper lip, and protruding lower lip. Fear raises the eyebrows, opens the eyes widely, and parts the lips slightly. In contrast, happiness crinkles the eye corners into crow's feet as the mouth raises into a smile. A neutral expression has relaxed, unfurrowed features with a closed mouth. Sadness raises the inner eyebrows while turning the lips downward into a frown. Surprise raises the eyebrows high, opens the eyes and mouth wide.Based on given face's characteristics, please identify the facial emotion of the image as one of the following: anger, contempt, disgust, fear, happiness, neutral, sadness, or surprise. Answer in one word."
        ),

        HumanMessage(image_documents=image_b64, content="Here is an image of a person's face")
        # AIMessage(content="disgust"),
        # AIMessage(content="happiness"),
        # AIMessage(content="fear"),
        # AIMessage(content="neutral"),
        # AIMessage(content="surprise"),
        # AIMessage(content="sadness")
    ]
     #HumanMessage(image_documents=image_b64, content="Here is an image of a person's face")
    
    prompt = ChatPromptTemplate.from_messages(messages)


    chain = prompt | llm | output_parser


    output_messages = chain.invoke({"input": prompt, "context": image_b64})

    return output_messages.strip("\n")


In [119]:
# Test for FER on JAFFE

CK_paths = find_data(CK=True, JAFFE=False, FER2013=False)

correct = 0
total = 0

angie = 0
disgust = 0
fe = 0
ha = 0
ne = 0
sa = 0
su = 0

# guesses[n][0] = correct label, guesses[n][1] = predicted label
guesses = []

emo_hat = classify_emotion(CK_paths[0][1], display=False)
print(CK_paths[0][0], emo_hat, emo_hat.lower())





{'anger': 45, 'contempt': 18, 'disgust': 59, 'fear': 25, 'happiness': 69, 'neutral': 593, 'sadness': 28, 'surprise': 83}
anger 0 0


In [108]:
# anger, disgust, fear, happiness, neutral, sadness, surprise
# 30 29 32 31 30 31 30
matrix = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
# column = correct, row= guessed
emo_dict = {"anger":0, "disgust":1, "fear":2, "happiness":3, "neutral":4, "sadness":5, "surprise":6}

for idx, [emo, file] in enumerate(JAFFE_paths):
    emo_hat = classify_emotion(file, display=False)
    if emo_hat == emo:
        correct += 1
    #guesses.append([emo.lower(), emo_hat.lower()])
    if emo_hat.lower() in emo_dict.keys():
        matrix[emo_dict[emo_hat.lower()]][emo_dict[emo]] +=1
    total +=1
    print(correct, total)
    
for i in matrix:
    print(i)
print(guesses)


        

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
2 33
3 34
3 35
3 36
3 37
4 38
4 39
4 40
4 41
4 42
4 43
4 44
4 45
4 46
4 47
4 48
4 49
4 50
5 51
5 52
5 53
5 54
6 55
6 56
6 57
6 58
6 59
6 60
6 61
6 62
6 63
6 64
6 65
6 66
6 67
6 68
6 69
6 70
6 71
7 72
7 73
7 74
7 75
7 76
7 77
7 78
7 79
7 80
7 81
7 82
7 83
7 84
7 85
7 86
7 87
7 88
7 89
7 90
7 91
7 92
7 93
7 94
7 95
7 96
7 97
7 98
7 99
7 100
7 101
7 102
7 103
7 104
7 105
7 106
7 107
7 108
7 109
7 110
7 111
7 112
7 113
7 114
7 115
7 116
7 117
7 118
7 119
7 120
7 121
7 122
7 123
8 124
8 125
8 126
8 127
8 128
8 129
8 130
8 131
8 132
8 133
8 134
8 135
8 136
8 137
8 138
8 139
8 140
8 141
8 142
8 143
8 144
8 145
8 146
8 147
8 148
8 149
8 150
8 151
8 152
8 153
8 154
8 155
8 156
8 157
8 158
9 159
9 160
9 161
9 162
9 163
9 164
9 165
9 166
9 167
10 168
10 169
10 170
10 171
10 172
10 173
10 174
10 175
10 176
10 177
10 178
10 179
10 180
11 181
11 182
1

1 1
1 2
2 3
2 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 12
5 13
5 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
6 25
6 26
6 27
6 28
6 29
6 30
6 31
6 32
7 33
8 34
8 35
8 36
8 37
9 38
9 39
9 40
9 41
9 42
9 43
10 44
11 45
11 46
11 47
11 48
11 49
11 50
11 51
11 52
11 53
11 54
11 55
11 56
11 57
11 58
11 59
12 60
12 61
13 62
13 63
13 64
13 65
13 66
13 67
13 68
13 69
14 70
14 71
15 72
15 73
15 74
15 75
15 76
15 77
15 78
15 79
15 80
15 81
15 82
15 83
15 84
15 85
15 86
15 87
15 88
15 89
15 90
15 91
15 92
15 93
15 94
15 95
15 96
15 97
15 98
15 99
15 100
15 101
15 102
15 103
15 104
15 105
15 106
15 107
15 108
15 109
15 110
15 111
15 112
15 113
15 114
15 115
15 116
16 117
16 118
16 119
17 120
17 121
17 122
17 123
18 124
18 125
18 126
18 127
18 128
18 129
18 130
18 131
19 132
19 133
19 134
19 135
19 136
20 137
20 138
20 139
20 140
20 141
20 142
20 143
20 144
20 145
20 146
20 147
20 148
20 149
20 150
20 151
20 152
20 153
20 154
20 155
20 156
20 157
20 158
21 159
21 160
21 161
21 162
21 163
21 164
21 165
21 166
21 167
22 168
22 169
22 170
22 171
23 172
23 173
23 174
23 175
23 176
23 177
23 178
23 179
23 180
23 181
23 182
23 183
23 184
23 185
23 186
23 187
23 188
23 189
23 190
23 191
23 192
23 193
23 194
23 195
23 196
23 197
23 198
23 199
23 200
23 201
23 202
23 203
23 204
23 205
24 206
24 207
24 208
24 209
24 210
24 211
24 212
24 213
[['anger', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['anger', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', ' anger'], ['anger', ' anger'], ['fear', ' anger'], ['fear', ' anger'], ['fear', ' anger'], ['fear', ' anger'], ['disgust', ' anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['anger', 'anger'], ['anger', 'anger'], ['fear', 'user: anger'], ['disgust', 'user: anger'], ['disgust', 'user: anger'], ['anger', 'user: anger'], ['fear', 'user: anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['disgust', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', ' angel madeline phipps 1902-1985'], ['fear', ' angel madeline phipps 1902-1985'], ['disgust', 'user: anger'], ['fear', 'user: anger'], ['disgust', 'user: anger'], ['fear', 'user: anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'user: anger'], ['fear', 'user: anger'], ['fear', 'user: anger'], ['fear', 'user: anger'], ['fear', 'user: anger'], ['fear', ' anger'], ['fear', ' anger'], ['disgust', ' anger'], ['fear', ' anger'], ['fear', ' anger'], ['fear', ' anger'], ['anger', 'user: surprise'], ['anger', 'user: surprise'], ['disgust', 'user: surprise'], ['fear', 'user: surprise'], ['fear', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['disgust', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['anger', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger'], ['fear', 'anger']]


from pydantic import BaseModel

from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.core.program import MultiModalLLMCompletionProgram
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core import SimpleDirectoryReader


GOOGLE_API_KEY = #MODEL_NAME = "models/gemini-pro-vision"

gemini_llm = GeminiMultiModal(
        api_key=GOOGLE_API_KEY,
        model_name=MODEL_NAME
    )

image_documents = SimpleDirectoryReader(
        input_files=[list_paths[0][1]]
    ).load_data()

class Analysis(BaseModel):
    """analysis questioning"""
    number_people: int
    describe_the_face: str
    eyes: str
    eyebrows: str
    mouth: str
    sentiment: str
#    Please_analyze_the_sentiment_of_person_Focus_on_identifying_the_emotion_based_on_the_facial_features_of_the_person_Avoid_subjective_opinions_or_commentary_unrelated_to_the_person_itself:str
    


llm_program = MultiModalLLMCompletionProgram.from_defaults(
        output_parser=PydanticOutputParser(output_cls=Analysis),
        image_documents=image_documents,
        prompt_template_str="Please analyze the sentiment of person. Focus on identifying the emotion based on the facial features of the person. Avoid subjective opinions or commentary unrelated to the person itself. Do not respond neutral.",
        multi_modal_llm=gemini_llm,
        verbose=True
    )

response = llm_program()